In [1]:
from langchain.text_splitter import MarkdownTextSplitter, RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import dotenv
dotenv.load_dotenv()
FAQ_CHROMA_PATH="chroma_data"

#load myiam
with open("documents/myiam.txt", "r") as f:
    text = f.read()

# Chunk the text
splitter = MarkdownTextSplitter(chunk_size=1500, chunk_overlap=50)
chunks = splitter.split_text(text)

# Merge headers with content
documents = []
for chunk in chunks: 
    header = chunk.split('\n')[0]  # Assuming header is followed by /n
    documents.append(Document(page_content=chunk, metadata={"header": header, "category": "MyIAM"}))


In [2]:
# load PO
with open("documents/product_owner.txt", "r") as f:
    text = f.read()

# Chunk the text
splitter = MarkdownTextSplitter(chunk_size=1024, chunk_overlap=50)
chunks = splitter.split_text(text)

# Merge headers with content
for chunk in chunks:  
    header = chunk.split('\n')[0]
    documents.append(Document(page_content=chunk, metadata={"header": header, "category": "PO"}))

Huggingfaceembeddings downloads the model the first time and then it uses it locally, but we want it in local from the get to go. You need sentence-transformers pip installed to use them.

For getting the model locally you can go to 
- [https://huggingface.co/sentence-transformers/all-mpnet-base-v2](https://huggingface.co/sentence-transformers/all-mpnet-base-v2)  Better quality but slower: 1m 32 seconds
- [https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)  Fast and lightweight: 4 segs

Click "Files and versions" tab
Download all the files to a local directory WITH GIT LFS 
```shell
./models/all-MiniLM-L6-v2/
├── config.json
├── pytorch_model.bin
├── tokenizer.json
├── tokenizer_config.json
├── vocab.txt
└── modules.json
```
### with sentence-transformers directly

In [3]:
from langchain_chroma import Chroma
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("./models/all-MiniLM-L6-v2")

# Create wrapper for LangChain
class LocalEmbeddings:
    def __init__(self, model):
        self.model = model
    
    def embed_documents(self, texts):
        return self.model.encode(texts, convert_to_tensor=False).tolist()
    
    def embed_query(self, text):
        return self.model.encode([text], convert_to_tensor=False)[0].tolist()

embeddings = LocalEmbeddings(model)

# Create vector store
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=FAQ_CHROMA_PATH
)

### with HuggingFaceEmbeddings

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

# Initialize local embeddings
embeddings = HuggingFaceEmbeddings(
    # model_name="sentence-transformers/all-MiniLM-L6-v2",  
    model_name="./models/all-MiniLM-L6-v2", #local path
    model_kwargs={'device': 'cpu'}  # or 'cuda' if you have GPU 
)

# Create vector store
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=FAQ_CHROMA_PATH
)